In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from functools import reduce
import ast
from unidecode import unidecode
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
pd.set_option("display.max_columns", None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

In [3]:
dfbarrios= pd.read_excel('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/distr_barrios_codpostal.xlsx')

In [4]:
dfi= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionSilvia23122024.csv',index_col=False)
#dfr_pm= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/redpiso_pisosmadrid_mejorado.csv')
dfpisos= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionAle29122024.csv')

In [6]:
dfpisos.columns


Index(['estado', 'url', 'distrito', 'barrio', 'cod_distrito', 'cod_barrio',
       'tipologia', 'detalle', 'consumo_energia', 'referencia',
       'fecha_publicacion', 'm2_utiles', 'ano_construccion', '€_comunidad_mes',
       'orientacion', 'jardin', 'aire_acondicionado', 'calefaccion', 'cocina',
       'tipo_suelo', 'piscina', 'armarios', 'garaje', 'terraza', 'trastero',
       'ascensor', 'balcon', 'title', 'ubicacion', 'precio', 'habitaciones',
       'banos', 'm2_constr', 'planta', 'EUR/m2', 'fecha_descarga', 'web',
       'interior', 'exterior'],
      dtype='object')

In [9]:
dfpisos["€_comunidad_mes"].unique()

array([nan, 'Entre 10 y 20 €', 'Entre 20 y 40 €', 'Entre 40 y 60 €',
       '290', '250', 'Entre 80 y 100 €', 'Entre 60 y 80 €',
       'Más de 100 €', '65', '200', '75€', '350  € aparte calefacción',
       '165€ limpieza, pisicina, gimnasio.....', '1.000', 'No tiene',
       '37', '205 incluido ibi que esta prorrateado cada mes', '113,60',
       '130 €', '50€ aprox.', 'Trimestral', '59', '50', '230 + 36',
       '125€ (116,37€-134,66€)', '45,50 €/mes',
       'Gastos comunidad: 170€ ibi: 1.400€', '31€/mes', '99', '99€',
       '250€', '139',
       'Cumunidad 615€ aprox. Incluye calefacción y agua .', '620 euros',
       '684€ incluida agua y calefacción', '358€+consumos',
       'Gastos comunidad 385€ ibi 1.500€', '440€', '364', '400€',
       '545€ incluida calefacción', '600€', '91.26€  ibi 482€', '230',
       'Comunidad: 320€ mensual  / ibi 1.040,96€ anual',
       'Calefaccion incluida', '189', '157 / ibi 924', '308',
       '680€ /ibi: 1.700€', '400', '500',
       'C. : 155 

In [403]:
dfp= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionAle29122024.csv')

In [404]:
dfp=dfp[["url","precio","tipologia"]]

In [405]:
dfp = dfp[dfp['tipologia'] != 'Finca']

In [411]:
dfp.query("precio.isnull()")

,url,precio,tipologia
135,https://www.pisos.com/comprar/atico-arcos28032...,NaN,Ático
731,https://www.pisos.com/comprar/estudio-casco_hi...,NaN,Estudio
882,https://www.pisos.com/comprar/piso-adelfas2800...,NaN,Piso
3461,https://www.pisos.com/comprar/piso-puerta_boni...,NaN,Piso
4736,NaN,NaN,NaN


In [333]:
dfp.shape

(4733, 3)

In [327]:
dfpisos.shape

(4733, 77)

In [358]:
dfpisos = dfpisos.merge(dfp[['url', 'piscina']], on='url', how='left')

In [79]:
dfb= pd.read_feather('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/df_distritos.ftr')

In [80]:
dfb.columns

Index(['cod_distrito', 'distrito', 'cod_barrio', 'barrio', 'distrito_n',
       'barrio_n'],
      dtype='object')

In [ ]:
#creo la columna tiene armario
dfpisos['tiene_armario'] = dfpisos['armarios'].apply(lambda x: 1 if pd.notna(x) and x != '' else np.nan)

In [63]:

def obtener_valores_por_columna(df, columna):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}




In [71]:
def procesar_distritos_nulos(df, dfbarrios):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    filas_nulas = df[df['distrito'].isna() & df['barrio'].notna()] 
    for idx, fila in filas_nulas.iterrows(): 
        valores = obtener_valores_por_columna(dfbarrios, 'nombre_barrio', fila['barrio']) 
        if valores:
             df.at[idx, 'cod_barrio'] = valores.get('cod_barrio') 
             df.at[idx, 'cod_distrito'] = valores.get('cod_distrito') 
             df.at[idx, 'distrito'] = valores.get('nombre_distrito')






In [90]:
# Función para normalizar texto (eliminar acentos y convertir a minúsculas)
def normalizar_texto(texto):
 return unidecode(texto).lower()

In [75]:
#completo distritos nulos con barrio informado
procesar_distritos_nulos(dfpisos, dfbarrios)


In [106]:
dfpisos.query("title=='Apartamento en venta en Jerónimos'")

,estado,url,distrito,barrio,cod_distrito,cod_barrio,tipologia,detalle,certificado_energetico,referencia,fecha_publicacion,m2_utiles,ano_construccion,€_comunidad_mes,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,terraza,trastero,ascensor,balcon,title,ubicacion,precio,habitaciones,banos,m2_constr,planta,EUR/m2,fecha_descarga,web,interior,exterior
72,NaN,https://www.pisos.com/comprar/apartamento-jero...,Retiro,Jerónimos,3,35,Apartamento,Engel & Völkers presenta la oportunidad de dis...,No indicado,Ref.: IF77045-I-00YEZ6-W-02V46K,2024-10-01,NaN,Más de 50 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,1,0,Apartamento en venta en Jerónimos,Jerónimos (Distrito Retiro. Madrid Capital) / ...,2800000.00,4.00,4.00,244.00,11475.00,11475.00,2024-10-19,pisos.com,0,0
73,NaN,https://www.pisos.com/comprar/apartamento-jero...,Retiro,Jerónimos,3,35,Apartamento,Engel & Völkers presenta este magnífico piso r...,No indicado,Ref.: IF77045-I-00YIHC-W-02V2MQ,2024-08-02,NaN,NaN,NaN,NaN,Privado,NaN,Central,NaN,NaN,NaN,1,NaN,1,1,1,0,Apartamento en venta en Jerónimos,Jerónimos (Distrito Retiro. Madrid Capital) / ...,3120000.00,3.00,3.00,275.00,11345.00,11345.00,2024-10-19,pisos.com,0,0


In [122]:
def asignar_zona_distrito_barrio(row):
    if row['zona'] == 'Sanchinarro':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'San Andrés':
        row['distrito'] = 'Villaverde'
        row['barrio'] = 'VillaverdeAlto,CascoHistóricodeVillaverde'
    if row['zona'] == 'Virgen del Cortijo':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'Montecarmelo':
        row['distrito'] = 'Fuencarral-El Pardo'
        row['barrio'] = 'Mirasierra'

    return row

In [ ]:


# Filtrar las filas donde 'distrito' o 'barrio' son nulos
filas_nulas = dfpisos[dfpisos['distrito'].isna() | dfpisos['barrio'].isna()]

for idx, fila in filas_nulas.iterrows():
    #bar = normalizar_texto(fila['ubicacion'].split()[:2])
   
    bar = normalizar_texto(fila['ubicacion'].split('(')[0].strip().replace('/', '')).strip()
    print(bar)
    if bar=='salvador':
        bar='elsalvador'
    if bar=='san andres':
        bar='villaverdealto,cascohistoricodevillaverde'
    if bar=='palos de moguer':
        bar='palosdemoguer'
    if bar=='virgen del cortijo-manoteras':
        bar='valdefuentes'  
    if bar=='sanchinarro':
        bar= 'valdefuentes'    
    if bar=='montecarmelo':
        bar='mirasierra'
    if bar=='ambroz':
        bar='cascohistoricodevicalvaro'
    #para los berrocales como es un barrio nuevo, lo creo aunque o exista oficialmente
    if bar=='los berrocales':
        dfpisos.at[idx, 'cod_barrio'] = 195
       
    valores = obtener_valores_por_columna(dfb, 'barrio_n', bar)
    if valores:
      
        if pd.isna(dfpisos.at[idx, 'barrio']): 
            dfpisos.at[idx, 'barrio'] = valores.get('barrio')       
        if pd.isna(dfpisos.at[idx, 'cod_barrio']): 
            dfpisos.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(dfpisos.at[idx, 'cod_distrito']): 
            dfpisos.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(dfpisos.at[idx, 'distrito']): 
            dfpisos.at[idx, 'distrito'] = valores.get('distrito')
        dfpisos.at[idx, 'zona'] = 'bar'

In [126]:
dfb.query("barrio=='Palos de Moguer'")

,cod_distrito,distrito,cod_barrio,barrio,distrito_n,barrio_n


In [174]:
#pongo los valores a pelo de cod_barrio y distrito que no tengo
dfpisos.query("referencia=='Ref.: 4510732-21249875'")

,estado,url,distrito,barrio,cod_distrito,cod_barrio,tipologia,detalle,certificado_energetico,referencia,fecha_publicacion,m2_utiles,ano_construccion,€_comunidad_mes,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,terraza,trastero,ascensor,balcon,title,ubicacion,precio,habitaciones,banos,m2_constr,planta,EUR/m2,fecha_descarga,web,interior,exterior,zona
272,A estrenar,https://www.pisos.com/comprar/atico-los_berroc...,Vicálvaro,NaN,NaN,195,Ático,"Portal 2, A C. Ático en una finca de obra nuev...",En trámite,Ref.: 4510732-21249875,2024-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,Ático en venta en los Berrocales,/ / /,498000.00,3.00,3.00,2.00,157.00,3171.00,2024-10-19,pisos.com,0,0,NaN


In [ ]:
# Filtrar las filas donde 'referencia' es igual a 'Ref.: 4510732-21249875' 
 #filas_referencia = dfpisos.query("referencia == 'Ref.: 4510732-21249875'")
#  # Asignar el valor 195 a la columna 'cod_barrio' en las filas filtradas
dfpisos.loc[dfpisos.query("referencia == 'Ref.: 4510732-21249875'").index, 'cod_barrio']=195


In [197]:
dfpisos.loc[dfpisos.query("referencia == 'Ref.: IF77095-LCM30872-1'").index, 'cod_barrio']=155
dfpisos.loc[dfpisos.query("referencia == 'Ref.: IF77095-LCM30872-1'").index, 'barrio']='San Pascual'

In [178]:
dfpisos.loc[dfpisos.query("referencia == 'Ref.: GM17-200378'").index, 'cod_barrio']=191
dfpisos.loc[dfpisos.query("referencia == 'Ref.: IF77045-I-00YN6P-W-02X311'").index, 'cod_barrio']=166
dfpisos.loc[dfpisos.query("referencia == 'Ref.: SA283-SIM1201071'").index, 'cod_barrio']=201

In [190]:
dfbarrios.tail(20)

,cod_distrito,nombre_distrito,cod_barrio,cod_barrio_2,nombre_barrio,codigo_Postal
111,17,Villaverde,175,5,Los Rosales,28021
112,18,Villa de Vallecas,181,1,Casco Histórico de Vallecas,28031
113,18,Villa de Vallecas,182,2,Santa Eugenia,28031
114,18,Villa de Vallecas,183,3,Ensanche de Vallecas,28031
115,19,Vicálvaro,191,1,Casco Histórico de Vicálvaro,28032
116,19,Vicálvaro,192,2,Valdebernardo,28032
117,19,Vicálvaro,193,3,Valderrivas,28032
118,19,Vicálvaro,194,4,El Cañaveral,28032
119,20,San Blas-Canillejas,201,1,Simancas,28037
120,20,San Blas-Canillejas,202,2,Hellín,28037


In [206]:
dfbarrios["nombre_distrito"] = dfbarrios["nombre_distrito"].apply(lambda x: x.strip())


In [ ]:
#teniendo codigo de barrio, pongo el resto de columnas sino tienen valor
# Filtrar las filas donde 'distrito' o 'barrio' son nulos
filas_nulas = dfpisos[dfpisos['distrito'].isna() | dfpisos['barrio'].isna()  | dfpisos['cod_distrito'].isna()]

for idx, fila in filas_nulas.iterrows():
    #bar = normalizar_texto(fila['ubicacion'].split()[:2])
   
    bar = (fila['distrito'])
    print(bar)
    fila = dfbarrios.loc[dfbarrios['nombre_distrito'] == bar]
    if not fila.empty:
       valores=fila.iloc[0].to_dict()
       print(valores)
       dfpisos.at[idx, 'cod_distrito'] = valores.get('cod_distrito')



In [210]:
dfi.shape,dfr_pm.shape,dfpisos.shape

((3150, 79), (3266, 51), (4733, 40))

In [49]:
dfi["web"]="Indomio.com"
dfi["fecha_descarga"]="2024-11-20"

In [313]:
nuevos_nombres ={'armario_empotrado': 'tiene_armario','title2':'detalle','piscina':'tiene_piscina'
 } 
dfi = dfi.rename(columns=nuevos_nombres)

In [51]:
nuevos_nombres ={'consumo_energia': 'certificado_energetico'
 } 
dfpisos = dfpisos.rename(columns=nuevos_nombres)


In [52]:
dfi['certificado_energetico'] = dfi['letrace'].apply(lambda x: 'Disponible' if pd.notna(x) and x != '' else 'No indicado')


In [53]:

# Supongamos que ya tienes tu DataFrame dfpisos
# Eliminar las filas donde 'tipologia' es 'finca'
dfpisos = dfpisos[dfpisos['tipologia'] != 'Finca']


In [442]:
# Convertir la columna 'habitaciones' a numérico, reemplazando valores no convertibles por NaN
dfpisos['habitaciones'] = pd.to_numeric(dfpisos['habitaciones'], errors='coerce')

# Asignar valores a 'dormitorios' cuando 'habitaciones' es menor que 20
dfpisos.loc[dfpisos['habitaciones'] < 20, 'dormitorios'] = dfpisos['habitaciones']

# Eliminar la columna 'habitaciones'
dfpisos = dfpisos.drop(columns=['habitaciones'])



In [216]:
dfpisos["distrito"] = dfpisos["distrito"].apply(lambda x: x.strip())

In [348]:
dfi.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'detalle', 'consumoce_ano', 'letrace', 'emisiones_co2',
       'tipologia', 'propiedad_completa', 'estado', 'amueblado', 'planta',
       'habitaciones', 'dormitorios', 'banos', 'balcon', 'terraza', 'garaje',
       'trastero', 'ascensor', 'porteria', 'calefaccion', 'ano_construccion',
       'plantas_edificio', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'usufructo',
       'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'tiene_armario', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'c

In [222]:
# Eliminar la primera columna 
dfi = dfi.iloc[:,1:]

In [359]:
dfpisos.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'detalle', 'consumoce_ano', 'letrace', 'emisiones_co2',
       'tipologia', 'propiedad_completa', 'estado', 'amueblado', 'planta',
       'habitaciones', 'dormitorios', 'banos', 'balcon', 'terraza', 'garaje',
       'trastero', 'ascensor', 'porteria', 'calefaccion', 'ano_construccion',
       'plantas_edificio', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'usufructo',
       'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'tiene_armario', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'c

In [401]:
dfpisos.query("precio.isnull()")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,usufructo,nuda_propiedad,derecho_superficie,sup_comercial,tipo_inmueble,tiene_armario,buhardilla,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,hidromasaje,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,read_all,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina
135,"Ático en venta en Calle de Aquitania, s/n",https://www.pisos.com/comprar/atico-arcos28032...,NaN,NaN,nan,NaN,69.00,San Blas-Canillejas,20.00,Arcos,204.00,NaN,nan,NaN,nan,nan,nan,Ático,NaN,A estrenar,nan,nan,1.0,NaN,1.0,0,0,nan,0.00,0,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,pisos.com,2024-08-19,NaN,nan
731,"Estudio en venta en Calle de Camarillas, 4",https://www.pisos.com/comprar/estudio-casco_hi...,NaN,NaN,nan,NaN,50.00,Barajas,21.00,Casco Histórico de Barajas,213.00,NaN,nan,NaN,nan,nan,nan,Estudio,NaN,En buen estado,nan,nan,1.0,NaN,1.0,0,0,nan,0.00,0,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,pisos.com,2024-08-19,NaN,nan
878,"Piso en venta en Calle Doctor Esquerdo, cerca ...",https://www.pisos.com/comprar/piso-adelfas2800...,NaN,NaN,nan,NaN,55.00,Retiro,3.00,Adelfas,32.00,NaN,nan,NaN,nan,nan,nan,Piso,NaN,En buen estado,nan,3.0,1.0,NaN,1.0,0,0,nan,0.00,0,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,pisos.com,2024-08-19,NaN,nan
3457,"Piso en venta en Calle del Real Betis, 11",https://www.pisos.com/comprar/piso-puerta_boni...,NaN,NaN,nan,NaN,104.00,Carabanchel,11.00,Puerta Bonita,115.00,NaN,nan,NaN,nan,nan,nan,Piso,NaN,A estrenar,nan,1.0,3.0,NaN,2.0,0,0,nan,0.00,0,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,pisos.com,2024-08-19,NaN,nan
4732,nan,nan,NaN,NaN,nan,9701.00,92.00,Salamanca,4.00,Lista,45.00,NaN,nan,NaN,nan,nan,nan,NaN,NaN,nan,nan,3.0,2.0,NaN,2.0,0,0,nan,0.00,0,nan,NaN,Más de 50 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Más de 100 €,NaN,NaN,pisos.com,2024-10-19,En trámite,nan


In [363]:
dfpisos.shape,dfi.shape

((4733, 78), (3150, 78))

In [360]:
dfi["piscina"]="-"

In [250]:
dfi = dfi.drop(columns=['ciudad'])
dfpisos = dfpisos.drop(columns=['ciudad'])

In [294]:
# Reemplazar "Sí" por 1 y "No" por 0 
dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})

C:\Users\extas\AppData\Local\Temp\ipykernel_21240\3458785875.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})


In [362]:
# Asegurarse de que ambos dataframes tengan las mismas columnas, rellenando con NaN 
dfpisos = dfpisos.reindex(columns=dfi.columns, fill_value=None) 
dfi = dfi.reindex(columns=dfpisos.columns, fill_value=None) 
# # Sumar las filas 

In [443]:
# Asegurarse de que ambos dataframes tengan las mismas columnas, rellenando con NaN 
dfpisos = dfpisos.reindex(columns=dfi.columns, fill_value=None) 
dfi = dfi.reindex(columns=dfpisos.columns, fill_value=None) 
# # Sumar las filas 

#dfi['precio'] = dfi['precio'].str.replace('.', '').str.replace(',', '')
numeric_columns = ['precio','cod_distrito','cod_barrio','trastero','tiene_piscina','buhardilla','usufructo','derecho_superficie'] # Añade aquí todas las columnas numéricas 
dfi[numeric_columns] = dfi[numeric_columns].astype(float) 
dfpisos[numeric_columns] = dfpisos[numeric_columns].astype(float)

non_numeric_columns = ['piscina','habitaciones','garaje','planta','banos','porteria','amueblado','ano_construccion','title', 'url', 'distrito', 'estado','descuento','calle','letrace','emisiones_co2','consumoce_ano','tipo_inmueble','€_comunidad_mes'] # Añade aquí todas las columnas no numéricas
dfi[non_numeric_columns] = dfi[non_numeric_columns].astype(str) 
dfpisos[non_numeric_columns] = dfpisos[non_numeric_columns].astype(str)

# # Sumar las filas 
df_union = pd.concat([dfi, dfpisos], ignore_index=True)
#df_union = dfi.add(dfpisos, fill_value=None)

In [354]:
dfpisos.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'detalle', 'consumoce_ano', 'letrace', 'emisiones_co2',
       'tipologia', 'propiedad_completa', 'estado', 'amueblado', 'planta',
       'habitaciones', 'dormitorios', 'banos', 'balcon', 'terraza', 'garaje',
       'trastero', 'ascensor', 'porteria', 'calefaccion', 'ano_construccion',
       'plantas_edificio', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'usufructo',
       'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'tiene_armario', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'c

In [388]:
df_union.shape

(7883, 78)

In [365]:
df_union.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'detalle', 'consumoce_ano', 'letrace', 'emisiones_co2',
       'tipologia', 'propiedad_completa', 'estado', 'amueblado', 'planta',
       'habitaciones', 'dormitorios', 'banos', 'balcon', 'terraza', 'garaje',
       'trastero', 'ascensor', 'porteria', 'calefaccion', 'ano_construccion',
       'plantas_edificio', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'usufructo',
       'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'tiene_armario', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'c

In [444]:
#df_union = df_union.drop(columns=['disponibilidad'])
df_union = df_union.drop(columns=['usufructo'])
df_union = df_union.drop(columns=['derecho_superficie'])
df_union = df_union.drop(columns=['buhardilla'])
df_union = df_union.drop(columns=['hidromasaje'])

In [445]:

# Reemplazar los valores según las indicaciones
df_union['piscina'] = df_union['piscina'].replace({
    '-nan': 'NaN',
    'NaN': 'NaN',
    '-Comunitaria': 'Comunitaria',
    '-Propia': 'Privada',
    '-Con salorium': 'Comunitaria',
    '-1 piscina de adultos y 1 piscina de niños': 'Comunitaria'
})



In [463]:
df_union['tiene_jardin'] = df_union['jardin'].apply(lambda x: 1 if pd.notnull(x) and x != 'Sin jardín' else (0 if x == 'Sin jardín' else None))

In [464]:
df_union['jardin'] = df_union['jardin'].replace({
    'Jardín privado': 'Privado',
    'Jardín comunitario': 'Comunitario',
    'Jardín privado y comunitario': 'Privado'
})


In [447]:
#Reemplazar todas las tipologías que comienzan con "Casa" por "Casa" 
df_union['tipologia'] = df_union['tipologia'].replace(r'^Casa.*', 'Casa', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Apartamento.*', 'Apartamento', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Caserío.*', 'Casa', regex=True)

In [449]:

# Eliminar las filas donde 'precio' es nulo 
df_union = df_union.dropna(subset=['precio'])

In [467]:
df_union.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'detalle', 'consumoce_ano', 'letrace', 'emisiones_co2',
       'tipologia', 'propiedad_completa', 'estado', 'amueblado', 'planta',
       'habitaciones', 'dormitorios', 'banos', 'balcon', 'terraza', 'garaje',
       'trastero', 'ascensor', 'porteria', 'calefaccion', 'ano_construccion',
       'plantas_edificio', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_v

In [457]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7878 entries, 0 to 7881
Data columns (total 74 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      7878 non-null   object 
 1   url                                        7878 non-null   object 
 2   precio                                     7878 non-null   float64
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  7878 non-null   object 
 5   EUR/m2                                     7860 non-null   float64
 6   m2_constr                                  7865 non-null   float64
 7   distrito                                   7878 non-null   object 
 8   cod_distrito                               7878 non-null   float64
 9   barrio                                     7757 non-null   object 
 10  cod_barrio                   

In [469]:
#Guardo la union de los dos datasets
df_union.to_csv("20241231_Union_Sil_Ale.csv",index=False)
df_union.to_feather("20241231_Union_Sil_Ale.ftr")

In [10]:
dfr_pm.query("plantas=='Plantas 8'")

,titulo,cant_dormitorios,cant_banos,espacio_metros_cuadrados,tipologia,piscina,garaje,jardin,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal,cantidad_visitas,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicion-estado,antiguedad,ascensor,orientacion,sistema_agua,sistema_calefaccion,tipo_suelo,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,tipo_fachada
2579,Piso en venta en CALLE NARVAEZ Ibiza Retiro Ma...,3.00,3.00,160.00,piso,NaN,NaN,NaN,2024-07-04,NaN,1600000.00,NaN,NaN,0.00,NaN,NaN,NaN,0.00,1.00,ibiza,retiro,3.00,34.00,4.00,28009.00,237.00,3.00,NaN,NaN,NaN,Estado Perfecto Estado,Ano de construccion 1959,1.00,NaN,NaN,NaN,NaN,0.00,Garajes 1,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,Plantas 8,Pisos 2,0.00,NaN


In [ ]:
#en la columna antiguedad de dfr_pm quitar el texto año Ano de construccion del valor
dfr_pm['antiguedad']=dfr_pm['antiguedad'].str.replace('Ano de construccion','')
dfr_pm['antiguedad']=dfr_pm['antiguedad'].str.replace('Antiguedad','')
dfr_pm["plantas_edificio"]=dfr_pm['plantas'].str.replace('Plantas','')
# Crear la nueva columna 'exterior' y asignar el valor 1 cuando 'vista zona' sea 'Exterior'
dfr_pm['exterior'] = dfr_pm['vista zona'].apply(lambda x: 1 if x == 'Exterior' else np.nan)
dfr_pm=dfr_pm.drop("vista zona",axis=1)
dfr_pm=dfr_pm.drop("plantas",axis=1)
# Diccionario con los nuevos nombres de las columnas
#voy a suponer que la superficie es superficie construida
nuevos_nombres ={'descripcion': 'title',
'espacio_metros_cuadrados': 'm2_constr',
'm_cuadrados_utiles':'m2_utiles',
'cant_dormitorios': 'dormitorios',
'cant_banos': 'banos' ,
'condicion-estado':'estado',
'sistema_calefaccion':'calefaccion',
'antiguedad' :'ano_construccion'} 
dfr_pm = dfr_pm.rename(columns=nuevos_nombres)
#de alguna manera calefaccion estaba duplicada, quito las columnas duplicadas
dfr_pm = dfr_pm.loc[:, ~dfr_pm.columns.duplicated()]
#quito las filas con precio nulo
dfr_pm=dfr_pm.query("precio.notna()")
# Asegurarse de que todos los nombres de las columnas sean cadenas de texto 
dfr_pm.columns = dfr_pm.columns.astype(str) 
# Eliminar espacios en blanco al principio o al final de los nombres de las columnas
dfr_pm.columns = dfr_pm.columns.str.strip()